In [1]:
import pandas as pd

from gaia_kepler import data, tgas_match

### Crossmatch the Kepler Eclipsing Binary Catalog
http://keplerebs.villanova.edu/

In [2]:
# Load EBs from the original Kepler mission
kebc = pd.read_csv('data/kebc.csv', comment='#')

# Load EBs from the K2 mission
k2ebc = pd.read_csv('data/k2ebc.csv', comment='#')

In [3]:
# Match to TGAS
kebc_in_tgas = tgas_match(kebc)
k2ebc_in_tgas = tgas_match(k2ebc)

print len(kebc_in_tgas)
print len(k2ebc_in_tgas)

324
145


In [4]:
# Save to CSV
kebc_in_tgas.to_csv('data/kebc_in_tgas.csv')
k2ebc_in_tgas.to_csv('data/k2ebc_in_tgas.csv')

### Crossmatch SB9 (spectroscopic binaries)
http://sb9.astro.ulb.ac.be/

http://vizier.cfa.harvard.edu/viz-bin/VizieR?-source=B/sb9

SB9 is comprised of three tables

* main: positions, magnitudes, and spectral types
* orbits: orbital parameters
* alias: other designations

I joined the main and orbits tables on VizieR, matching by system number (Seq). Some systems have multiple orbits, so take the orbit with the best grade. grade 0 = poor, grade 5 = definitive.

I'm using the decimal RA and DEC computed by VizieR in FK5.

In [10]:
sb9 = pd.read_csv('data/sb9_join.tsv', delimiter='|', comment='#')

In [11]:
# Only keep systems with orbit grades >= 3
sb9 = sb9[sb9['Grade'].astype(int) >= 3]

In [12]:
# For duplicates, keep orbits with the higher grade
sb9 = sb9.sort_values(['Seq', 'Grade'], ascending=[True, False])
sb9 = sb9.drop_duplicates(subset='Seq')

In [13]:
sb9_in_tgas = tgas_match(sb9)
print len(sb9_in_tgas)

721


In [15]:
sb9_in_tgas.to_csv('data/sb9_in_tgas.csv')